# <font color='056938'> **Librerias** </font>

In [1]:
import numpy as np
import pandas as pd
import sqlite3 as sql
import plotly.graph_objs as go ### para gráficos
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from ipywidgets import interact ## para análisis interactivo
from sklearn import neighbors ### basado en contenido un solo producto consumido
import joblib
from sklearn.neighbors import NearestNeighbors

# <font color='056938'> **Cargar base de datos** </font>

In [ ]:
# Conexion con Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Cambio de directorio de Trabajo
import os
import sys
path = '/content/drive/MyDrive/Mod2/ANALITICA3'
os.chdir(path)
sys.path.append(path)

In [ ]:
import sqlite3 as sql
import pandas as pd
import numpy as np

conn=sql.connect('/content/drive/MyDrive/Mod2/ANALITICA3/data/db_movies')
cur=conn.cursor() ###para funciones que ejecutan sql en base de datos

In [ ]:
### para verificar las tablas que hay disponibles
cur.execute("SELECT name FROM sqlite_master where type='table' ")
cur.fetchall()

In [ ]:
# Cargas la tabla que ya contiene los datos procesados
tabla_completa = pd.read_sql_query("SELECT * FROM full_ratings", conn)

tabla_completa

In [ ]:
tabla_completa.info()#Se confirma que año sea de tipo entero

# <font color='056938'> **Sistema de recomendación basado en contenido** </font>

In [ ]:
#se escala el año para que este en el mismo rango
sc = StandardScaler()
tabla_completa[["year_sc"]]=sc.fit_transform(tabla_completa[['movie_year']])

Este codigo de abajo es provisional mientras averiguo como traer la que monte en el sql :)

In [ ]:
# Separar los géneros por el carácter '|' y expandirlos en múltiples filas, se hace de esta manera ya que sqlite no tiene esa funcion SALE EN FILAS
df_genres = tabla_completa.assign(movie_genres=tabla_completa['movie_genres'].str.split('|')).explode('movie_genres')

# Renombrar para claridad
df_genres = df_genres.rename(columns={'movie_genres': 'genre'})

df_genres

In [ ]:
#sacar las filas que no se van a utilizar
movie_dum1 = df_genres.drop(columns = ['user_id', 'movie_rating', 'movie_timestamp', 'movie_year', 'movie_title'])
movie_dum1

In [ ]:
movie_dum1['genre'].nunique()# se verifica cuantos generos únicos hay

In [ ]:
#se dumiza los generos
col_dum = ['genre']
movie_dum2 = pd.get_dummies(movie_dum1, columns = col_dum)
movie_dum2

In [ ]:
#Al quedar varios registros por filas de la misma pelicula pero todas con genero diferente se agrupan para disminuir la cantidad de datos a procesar a solamente un registro de pelicula
movie_unificado = movie_dum2.groupby('movie_id', as_index=False).max()
movie_unificado

In [ ]:
#se define el modelo knn con 10 recomendaciones, se usa la métrica coseno
model = neighbors.NearestNeighbors(n_neighbors=10, metric='cosine')

In [ ]:
#se entrena el modelo
model.fit(movie_unificado)

In [ ]:
#se definen los arrays de distancia(en terminos del coseno) y los ids para saber que pelicula es
dist, idlist = model.kneighbors(movie_unificado)

In [ ]:
#distancias mas cercanas, entre mas cercano a cero mas se parecen las peliculas, esas son las que se van a recomendar
distancias = pd.DataFrame(dist)
distancias

In [ ]:
#Para saber a que id de pelicula es
id_list = pd.DataFrame(idlist)#
id_list

In [ ]:
#función interactiva para consultar las recomendaciones
def MovieRecommender(movie_name = list(tabla_completa['movie_title'].value_counts().index)):
    movie_list_name = []
    movie_id = tabla_completa[tabla_completa['movie_title'] == movie_name].index
    movie_id = movie_id[0]
    for newid in idlist[movie_id]:
        movie_list_name.append(tabla_completa.loc[newid].movie_title)
    return movie_list_name

In [ ]:
print(interact(MovieRecommender))

In [ ]:
import nbformat

# Path to your notebook
input_notebook = '/content/drive/MyDrive/Mod2/ANALITICA3/Trabajo Marketing/Modelo_2.ipynb'
output_notebook = '/content/drive/MyDrive/Mod2/ANALITICA3/Trabajo Marketing/3.Modelo_2.ipynb'

# Load the notebook
with open(input_notebook, 'r') as f:
    notebook = nbformat.read(f, as_version=4)

# Check if the notebook has 'metadata.widgets' and remove it
if 'widgets' in notebook.metadata:
    del notebook.metadata['widgets']  # Removes widgets metadata completely

# Save the modified notebook
with open(output_notebook, 'w') as f:
    nbformat.write(notebook, f)

print(f"Fixed notebook saved as {output_notebook}")